In [8]:
import torch
from torch.utils.data import Dataset
import json
import os
import torch.nn as nn
import torch.nn.functional as func
from torch.utils.data import DataLoader
import torch.optim as optim
from os.path import expanduser
import splitfolders
import shutil
import glob
import numpy as np
from sklearn.model_selection import train_test_split

In [9]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# torch.cuda.set_per_process_memory_fraction(0.9, 0)
print(device)

cuda


In [10]:
class KpVelDataset(Dataset):
    def __init__(self, json_folder):
        super(KpVelDataset, self).__init__()
        self.data = []
        for json_file in sorted(os.listdir(json_folder)):
            if json_file.endswith('_combined.json'):
                with open(os.path.join(json_folder, json_file), 'r') as file:
                    data = json.load(file)
                    start_kp = data['start_kp']
                    next_kp = data['next_kp']
                    velocity = data['velocity']
                    self.data.append((start_kp, next_kp, velocity))

    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        start_kp, next_kp, velocity = self.data[idx]
        # Extract and flatten the first two elements of each keypoint in start_kp
        start_kp_flat = torch.tensor([kp for sublist in start_kp for kp in sublist[0][:2]], dtype=torch.float)
        # Extract and flatten the first two elements of each keypoint in next_kp
        next_kp_flat = torch.tensor([kp for sublist in next_kp for kp in sublist[0][:2]], dtype=torch.float)
        velocity = torch.tensor(velocity)
        return start_kp_flat, next_kp_flat, velocity

In [11]:
def train_test_split(src_dir):
#     dst_dir_img = src_dir + "images"
    dst_dir_anno = src_dir + "annotations"
    
    if os.path.exists(dst_dir_anno):
        print("folders exist")
    else:
        os.mkdir(dst_dir_anno)
        
#     for jpgfile in glob.iglob(os.path.join(src_dir, "*.jpg")):
#         shutil.copy(jpgfile, dst_dir_img)

    for jsonfile in glob.iglob(os.path.join(src_dir, "*_combined.json")):
        shutil.copy(jsonfile, dst_dir_anno)
        
    output = src_dir + "split_folder_reg"
    
    splitfolders.ratio(src_dir, # The location of dataset
                   output=output, # The output location
                   seed=42, # The number of seed
                   ratio=(0.8, 0.1, 0.1), # The ratio of split dataset
                   group_prefix=None, # If your dataset contains more than one file like ".jpg", ".pdf", etc
                   move=False # If you choose to move, turn this into True
                   )
    
#     shutil.rmtree(dst_dir_img)
    shutil.rmtree(dst_dir_anno)
    
    return output  

In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class KeypointRegressionNet(nn.Module):
    def __init__(self):
        super(KeypointRegressionNet, self).__init__()
        # Define the architecture
        self.fc1 = nn.Linear(15, 512)  # 18 keypoints + 3 velocity values
        self.fc2 = nn.Linear(512, 512)
        self.fc3 = nn.Linear(512, 256)
        self.fc4 = nn.Linear(256, 128)
        self.fc5 = nn.Linear(128, 12)  # Output size is 18 (6 keypoints * 3 values each)

    def forward(self, start_kp, velocity):
        # Flatten start keypoints and concatenate with velocity
        x = torch.cat((start_kp, velocity), dim=1)
        
        # Forward pass through the network
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = self.fc5(x)     # No activation function in the last layer
        return x

In [ ]:
# Initialize dataset and data loader
# to generalize home directory. User can change their parent path without entering their home directory
num_epochs = 100
batch_size = 64
v = 2
layers = 5
root_dir = '/home/jc-merlab/Pictures/panda_data/panda_sim_vel/regression_dataset_panda/'
# print(root_dir)
split_folder_path = train_test_split(root_dir)
dataset = KpVelDataset(root_dir)
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Initialize model
model = KeypointRegressionNet()  # Adjust input_size as necessary
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# Training loop
for epoch in range(num_epochs):
    for start_kp, next_kp, velocity in data_loader:
        optimizer.zero_grad()
        velocity = velocity.squeeze(1)
        # print(start_kp.shape)
        # print(velocity.shape)
        output = model(start_kp, velocity)
        loss = criterion(output, next_kp)
        loss.backward()
        optimizer.step()
        # print("output", output)
    print(f'Epoch {epoch+1}, Loss: {loss.item()}')
    
# # Save the trained model
model_save_path = f'/home/jc-merlab/Pictures/Data/trained_models/reg_nkp_panda_b{batch_size}_e{num_epochs}_v{v}_l{layers}.pth'
torch.save(model.state_dict(), model_save_path)

# model_save_path = f'/home/jc-merlab/Pictures/Data/trained_models/reg_nkp_b{batch_size}_e{num_epochs}_v{v}.pth'
# torch.save({
#     'model_state_dict': model.state_dict(),
#     'model_structure': KeypointRegressionNet()
# }, model_save_path)
# print(f"Model saved to {model_save_path}")


Copying files: 4837 files [00:00, 4948.17 files/s]


Epoch 1, Loss: 5865.19873046875
Epoch 2, Loss: 1429.5460205078125
Epoch 3, Loss: 1336.2943115234375
Epoch 4, Loss: 1451.6083984375
Epoch 5, Loss: 1139.1907958984375
Epoch 6, Loss: 1109.41845703125
Epoch 7, Loss: 776.5753173828125
Epoch 8, Loss: 387.30633544921875
Epoch 9, Loss: 389.4540710449219
Epoch 10, Loss: 252.189453125
Epoch 11, Loss: 178.06570434570312
Epoch 12, Loss: 109.74031066894531
Epoch 13, Loss: 163.2052764892578
Epoch 14, Loss: 88.64380645751953
Epoch 15, Loss: 57.750396728515625
Epoch 16, Loss: 36.22111892700195
Epoch 17, Loss: 33.30924606323242
Epoch 18, Loss: 26.322668075561523
Epoch 19, Loss: 20.82076072692871
Epoch 20, Loss: 17.409894943237305
Epoch 21, Loss: 17.210865020751953
Epoch 22, Loss: 12.044438362121582
Epoch 23, Loss: 10.970352172851562
Epoch 24, Loss: 10.80113410949707
Epoch 25, Loss: 9.580595016479492
Epoch 26, Loss: 9.018709182739258
Epoch 27, Loss: 9.288033485412598
Epoch 28, Loss: 7.555144786834717
Epoch 29, Loss: 8.051301002502441
Epoch 30, Loss: 6.4

In [ ]:
def test_model(model_path, test_data_dir):
    # Load the test dataset
    test_dataset = KpVelDataset(test_data_dir)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

    # Initialize the model and load the saved state
    model = KeypointRegressionNet()
    model.load_state_dict(torch.load(model_path))
    model.eval()

    # Criterion for evaluation
    criterion = nn.MSELoss()
    total_loss = 0

    # No gradient needed for evaluation
    with torch.no_grad():
        for start_kp, next_kp, velocity in test_loader:
            output = model(start_kp, velocity)
            for i in range(start_kp.size(0)):
                individual_start_kp = start_kp[i]
                individual_next_kp = next_kp[i]
                individual_velocity = velocity[i]
                predicted_next_kp = output[i]

                print("Start KP:", individual_start_kp)
                print("Actual Next KP:", individual_next_kp)
                print("Velocity:", individual_velocity)
                print("Predicted Next_kp:", predicted_next_kp)
                print("-----------------------------------------")
            loss = criterion(output, next_kp)
            total_loss += loss.item()
            
    # Calculate the average loss
    avg_loss = total_loss / len(test_loader)
    print(f'Average Test Loss: {avg_loss}')

# Usage
model_path = '/home/jc-merlab/Pictures/Data/trained_models/reg_nkp_panda_b64_e100_v2_l5.pth'  # Update with your model path
test_data_dir = '/home/jc-merlab/Pictures/panda_data/panda_sim_vel/regression_dataset_panda/split_folder_reg/test/annotations/'  # Update with your test data path
test_model(model_path, test_data_dir)